TODO: Fix compression agent, retry this approach.

In [1]:
import os
import sys
import json
import openai
import autogen
import tempfile
import chromadb
from pathlib import Path

import autogen
from autogen import AssistantAgent
from autogen import UserProxyAgent
from autogen.agentchat.contrib.compression_agent import CompressionAgent


# config_list = autogen.config_list_from_dotenv()
# Start logging
autogen.ChatCompletion.start_logging()


import tempfile
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())

env_var = [
    {
        'model': 'gpt-4',
        'api_key': os.getenv('OPENAI_API_KEY')
    },
    {
        'model': 'gpt-3.5-turbo',
        'api_key': os.getenv('OPENAI_API_KEY')
    }
]

# Create a temporary file
# Write the JSON structure to a temporary file and pass it to config_list_from_json
with tempfile.NamedTemporaryFile(mode='w+', delete=True) as temp:
    env_var = json.dumps(env_var)
    temp.write(env_var)
    temp.flush()

    # Setting configurations for autogen
    config_list = autogen.config_list_from_json(
        env_or_file=temp.name,
        filter_dict={
            "model": {
                "gpt-4",
                "gpt-3.5-turbo",
            }
        }
    )

assert len(config_list) > 0 
print("models to use: ", [config_list[i]["model"] for i in range(len(config_list))])


models to use:  ['gpt-4', 'gpt-3.5-turbo']


## Automata Art

In [ ]:
GPU_ACCEL_SYSTEM_PROMPT = """
You are an astute GPU Accelorator LLM Agent with a strong emphasis on optimization and efficiency in coding practices, especially in enabling GPU-accelerated computing when viable. 
You diligently inspect system settings upon user requests and foster a collaborative interaction with the Coding Assistant Agent to elucidate optimized coding strategies and library selections.

Objective:
- Incisive System Inspection: Deep-dive into hardware and software configurations, precisely identifying aspects like GPU capabilities, RAM size, CPU type and the operating system to glean opportunities for code optimization.
- Advise on Coding Strategies: Accurately communicate the detailed system information to the Coding Assistant Agent to derive coding advice, particularly focusing on identifying opportunities for GPU-accelerated computing and optimizing code execution.
- Uphold Security and Privacy: Vigilantly maintain the highest standards of data privacy and security during all phases of system inspection and interaction.
- Efficient Communication: Clearly and concisely present findings and information, ensuring an expedited and insightful response to user queries.

Workflow:
- Decode User Queries: Intuitively interpret user requests concerning system specifications and coding requirements, with a keen eye on extracting opportunities for optimization.
- Meticulous System Inspection: Investigate system specifics and assess any bottleneck issues or prospects that might enhance or impede optimized code execution.
- Engage with Coding Assistant: Relay detailed system information and collaborate to comprehend whether specific coding tasks, such as GPU-accelerated data processing, are feasible and to discern what code/libraries are optimal.

Remember:
- Deliver articulate, insightful, and expedited responses.
- Unwaveringly prioritize user privacy and data security.
- Enable the Coding Assistant Agent to formulate and provide profoundly optimized coding advice, rooted in real-time and meticulous system details.
- If it appear that no GPU is available, insist on other methods to find out if there is one available. Don't give up after one attempt - GPU acceloration is paramount.
- You should first check the type of system it is, and then decide how to best find out GPU details for acceloration of code.

Example Interaction:
User: "Can my system efficiently handle GPU-accelerated data processing using Python?"
You: Execute thorough system inspection -> Engage and Collaborate with Coding Assistant -> Provide user with a comprehensive and insightful response, inclusive of executable code when relevant.

Once you have assisted the coding assistant on GPU acceloration informaionyou should back out of the conversation, and let the others do their job. 
"""


REQUEST = """
Your task is to develop code to create beautiful and colourful cellular automata art. Use what ever tool our programming language you believe to be the most optimal.
You will make a video thats 100 seconds long. Make sure the patterns look very interesting and awe inspiring.

Run the code for me and save it to ./art.mp4.
Say only what is necessary to complete the tasks, don't send massive paragraphs to eachother.
Lastly, create highly efficient and well thought-out code to generate the art.
Make sure the video is playable and not corrupted. The MP4 should be runnable on a macbook.
"""

In [10]:
coding_assistant = AssistantAgent(
    name="coding_assistant",
    llm_config={
        "request_timeout": 1000,
        "seed": 42,
        "config_list": config_list,
        "temperature": 0.05,
    },
)

# gpu_accelorator_assistant = AssistantAgent(
#     name="gpu_accelorator_assistant",
#     system_message=GPU_ACCEL_SYSTEM_PROMPT,
#     llm_config={
#         "request_timeout": 1000,
#         "seed": 42,
#         "config_list": config_list,
#         "temperature": 0.5,
#     },
# )

compression_agent = CompressionAgent(
    name="compression_agent",
    is_termination_msg = lambda x: x.get("function_call", "").rstrip().endswith("TERMINATE"),
    llm_config={
        "request_timeout": 1000,
        "seed": 42,
        "config_list": config_list,
        "temperature": 0.05,
    },
)

coding_runner = UserProxyAgent(
    name="coding_runner",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=30,
    is_termination_msg = lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False
    },
)

# Create a group chat with agents and messages
# groupchat = autogen.GroupChat(agents=[coding_runner, coding_assistant, gpu_accelorator_assistant, compression_agent], messages=[])
groupchat = autogen.GroupChat(agents=[coding_runner, coding_assistant, compression_agent], messages=[])
# Create a GroupChatManager
manager = autogen.GroupChatManager(groupchat=groupchat)
# Define a task message

req = {
    'content': REQUEST
}

# Initiate the chat with the task message
coding_runner.initiate_chat(manager, message=req)
# coding_runner.send(recipient=coding_assistant, 
#                        message=request)

coding_runner (to chat_manager):


Your task is to develop code to create beautiful and colourful cellular automata art. Use what ever tool our programming language you believe to be the most optimal.
You will make a video thats 100 seconds long. Make sure the patterns look very interesting and awe inspiring.

Run the code for me and save it to ./art.mp4.
Say only what is necessary to complete the tasks, don't send massive paragraphs to eachother.
Lastly, create highly efficient and well thought-out code to generate the art.
Make sure the video is playable and not corrupted. The MP4 should be runnable on a macbook.


--------------------------------------------------------------------------------
coding_assistant (to chat_manager):

To create a cellular automata art video, we can use Python with the libraries numpy, matplotlib, and opencv. Here's a plan:

1. We will use numpy to create and manipulate a 2D array representing our cellular automata.
2. We will use matplotlib to create ima

ValueError: Message can't be converted into a valid ChatCompletion message. Either content or function_call must be provided.

In [ ]:
# coding_runner.send(recipient=coding_assistant, 
#                     message="Make another one thats it 20 seconds this time")

In [ ]:
from IPython.display import HTML
from base64 import b64encode

# Load the video
video_path = "./coding/art.mp4"  # Update with the path to your video file

# Encode the video
video_encoded = b64encode(open(video_path, "rb").read()).decode('ascii')

# Create HTML content
video_tag = f'''
<video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{video_encoded}" type="video/mp4">
</video>'''

# Display the video
HTML(data=video_tag)

